In [1]:
%cd ..

/mnt/c/Users/maxli/OneDrive - Van Oord/Documents/thesis/code


In [2]:
import pandas as pd
import geopandas as gpd
from glob import glob
import seaborn as sns
from atl_module.plotting import error_lidar_pt_vs_truth_pt,plot_photon_map,plot_tracklines_overview
from sklearn.metrics import r2_score
import numpy as np
from atl_module.geospatial_utils.raster_interaction import query_raster,query_from_lines

In [ ]:
aoidflist = []
for aoifile in glob('../data/test_sites/*/AOI.gpkg'):
    # get the name of each site
    sitename = aoifile.split('/')[3]
    df = gpd.read_file(aoifile)
    df['site']=sitename
    aoidflist.append(df)
    
aoi_combined = pd.concat(aoidflist)


In [ ]:
secchi_pt_list= []
for file in glob('../data/test_sites/*/secchi_pts.gpkg'):
    # get the name of each site
    sitename = file.split('/')[3]
    df = gpd.read_file(file)
    df['site']=sitename
    secchi_pt_list.append(df)
    
secchi_pt_combined = pd.concat(secchi_pt_list).reset_index(drop=True)

secchi_pt_combined = secchi_pt_combined.assign(X=secchi_pt_combined.geometry.x,Y=secchi_pt_combined.geometry.y)

In [3]:
# tracklines_dflist = [gpd.read_file(file) for file in glob('../data/test_sites/*/tracklines')]

tracklinesdflist = []
for file in glob('../data/test_sites/*/tracklines'):
    # get the name of each site
    sitename = file.split('/')[3]
    df = gpd.read_file(file)
    df['site']=sitename
    tracklinesdflist.append(df)
    
tracklines_combined = pd.concat(tracklinesdflist).reset_index(drop=True)


In [ ]:
tracklines_combined

In [6]:
interp_points = np.linspace(0,1,200)

line = tracklines_combined.geometry[0]
line_point_list = [line.interpolate(fraction,normalized=True) for fraction in interp_points]
xpoints = [point.x for point in line_point_list]
ypoints = [point.y for point in line_point_list]

df = pd.DataFrame({"X":xpoints,"Y":ypoints})
from atl_module.geospatial_utils.raster_interaction import query_raster

print(query_raster(df,'../data/test_sites/big_island/in-situ-DEM/truth.vrt'))

[6.3121395111084, 6.22054290771484, 6.13508319854736, 5.93313837051392, 6.00024318695068, 5.97113418579102, 10.5731658935547, 11.6974954605103, 11.7275915145874, 10.2527008056641, 8.61267566680908, 8.17540454864502, 6.85472917556763, 5.90798044204712, 6.47728204727173, 7.95343589782715, 8.8941764831543, 9.29537010192871, 9.39352703094482, 9.39365482330322, 9.40244483947754, 9.40257167816162, 9.4026985168457, 9.40282535552979, 9.51572132110596, 9.59680652618408, 9.2241096496582, 9.05634880065918, 8.81542682647705, 8.61695194244385, 8.51882839202881, 8.5090160369873, 8.18749523162842, 7.68021202087402, 7.34809541702271, 6.69640445709229, 6.36734104156494, 6.28194713592529, 6.2074556350708, 6.12779235839844, 6.05902004241943, 6.08176136016846, 6.01813745498657, 6.13130807876587, 5.96999406814575, 5.92264890670776, 5.90067434310913, 5.90369987487793, 5.89364957809448, 5.99273014068604, 5.63175487518311, 5.66159772872925, 5.73489475250244, 5.85992383956909, 6.69456386566162, 7.0873422622680

In [5]:
test = query_from_lines(line,'../data/test_sites/big_island/in-situ-DEM/truth.vrt')
print(test)

[6.3121395111084, 6.22054290771484, 6.13508319854736, 5.93313837051392, 6.00024318695068, 5.97113418579102, 10.5731658935547, 11.6974954605103, 11.7275915145874, 10.2527008056641, 8.61267566680908, 8.17540454864502, 6.85472917556763, 5.90798044204712, 6.47728204727173, 7.95343589782715, 8.8941764831543, 9.29537010192871, 9.39352703094482, 9.39365482330322, 9.40244483947754, 9.40257167816162, 9.4026985168457, 9.40282535552979, 9.51572132110596, 9.59680652618408, 9.2241096496582, 9.05634880065918, 8.81542682647705, 8.61695194244385, 8.51882839202881, 8.5090160369873, 8.18749523162842, 7.68021202087402, 7.34809541702271, 6.69640445709229, 6.36734104156494, 6.28194713592529, 6.2074556350708, 6.12779235839844, 6.05902004241943, 6.08176136016846, 6.01813745498657, 6.13130807876587, 5.96999406814575, 5.92264890670776, 5.90067434310913, 5.90369987487793, 5.89364957809448, 5.99273014068604, 5.63175487518311, 5.66159772872925, 5.73489475250244, 5.85992383956909, 6.69456386566162, 7.0873422622680

# Concatenate all bathymetry points from all sites

In [ ]:
dflist = []
for file in glob('../data/test_sites/*/all_bathy_pts.gpkg'):
    # get the name of each site
    sitename = file.split('/')[3]
    df = gpd.read_file(file)
    df['site']=sitename
    dflist.append(df)

bathy_pts_df = pd.concat(dflist).reset_index(drop=True).dropna()

## Some stats about all the total batymetry points found

do we see a spike around 2.3 or 4.2 due to ringing?


In [ ]:
# set bin edges to encapsulate the location where ringing occurs
binedges = np.arange(-25.05,0.6,0.1)
# 2.3 or 4.2 m

In [ ]:
ax = bathy_pts_df.sf_elev_MSL.plot.hist(bins=binedges,histtype='step',density=True,label='ICESat-2 Estimate',figsize=(20,10))
bathy_pts_df.true_elevation.plot.hist(ax=ax,bins=binedges,histtype='step',density=True,label='Actual Value')
# ax.set_xlim((-25,0))
ax.axvline(-2.3)
ax.axvline(-4.2)
ax.legend(loc='upper left')


In [ ]:
bathy_pts_df.groupby('site').agg({'error':lambda x:np.sqrt(np.mean(np.square(x))),'error_abs':'mean'}).sort_values('error').round(2)

based on the graph above we do not see significant deviation from the overall distribution of true bathymetry.

# Sites by Secchi Depth

In [ ]:
secchi_pt_combined = secchi_pt_combined.assign(gebco_elev = query_raster(secchi_pt_combined.assign(X=secchi_pt_combined.geometry.x,Y=secchi_pt_combined.geometry.y),'../data/GEBCO/GEBCO_2021_sub_ice_topo.nc'))
# mask out values based on GEBCO elevation 
secchi_pt_combined = secchi_pt_combined[secchi_pt_combined.gebco_elev<0]

In [ ]:
zsd_date_beam = secchi_pt_combined.dropna().groupby(['site',secchi_pt_combined.date,'beam']).mean().reset_index()

In [ ]:
secchi_pt_combined.loc[:,['site','zsd','sigma_zsd']].dropna().groupby('site').describe()

In [ ]:
ax = secchi_pt_combined.boxplot(column='zsd',by='site')
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)

In [ ]:
secchi_pt_combined.groupby('site').median()

# Analysis of tracklines by site

In [ ]:
tracklines_combined.groupby(['site']).mean()

# analyzing all bathymetry points from all sites

In [ ]:
# normalize the datetimes so that they can both match each other
bathy_pts_df = bathy_pts_df.assign(date=pd.to_datetime(bathy_pts_df.delta_time).dt.date)
zsd_date_beam = zsd_date_beam.assign(date=pd.to_datetime(zsd_date_beam.date).dt.date)


In [ ]:
joined_df = bathy_pts_df.merge(zsd_date_beam,how='outer',on=['beam','date','site']).drop(columns=['X_y','Y_y','gebco_elev_y'])

joined_df = joined_df.assign(depth_ph = joined_df.Z_geoid - joined_df.sea_level_interp)
joined_df

In [ ]:
joined_df = joined_df[joined_df.error_abs<5]

In [ ]:
joined_df.groupby(['date','beam']).count()

It is possible that for each site there will be a 

In [ ]:
tracklines_averaged = joined_df.groupby(['date','beam']).mean()
tracklines_averaged

In [ ]:
joined_df.corr('spearman')['error_abs'].sort_values()

In [ ]:
tracklines_averaged.plot.scatter(x='land_sig_conf',y='error_abs')

In [ ]:
tracklines_averaged.plot.scatter(x='oc_hconf_perc',y='error_abs')

In [ ]:
tracklines_averaged[tracklines_averaged.error_abs<5].plot.scatter(x='true_elevation',y='error_abs')

In [ ]:
ax = joined_df.plot.scatter(x='kde_val',y='error_abs',alpha=0.01)
# ax.axvline(0.15)

In [ ]:
joined_df.groupby('site').mean().plot.scatter(x='zsd',y='error_abs')

In [ ]:
joined_df.plot.scatter(x='sf_elev_MSL',y='true_elevation')

In [ ]:
# are estimates from areas below the secchi depth worse?
joined_df.groupby(-1*joined_df.sf_elev_MSL < (joined_df.zsd+10)).median()

In [ ]:
joined_df.assign(d_below_zsd=(-1*joined_df.sf_elev_MSL) - joined_df.zsd).plot.scatter(x='d_below_zsd',y='error_abs')

In [ ]:
errordict = {'RMSE':df.error.pow(2).mean()**0.5,'R2 Score':r2_score(df.true_elevation,df.sf_elev_MSL)}
ax = error_lidar_pt_vs_truth_pt(df,'All sites combined',errordict)


In [ ]:
ax.a

In [ ]:
df = df[df.true_elevation<0]
errordict = {'RMSE':df.error.pow(2).mean()**0.5,'R2 Score':r2_score(df.true_elevation,df.sf_elev_MSL)}
ax= error_lidar_pt_vs_truth_pt(df,'All Sites Excluding mountains',errordict)
ax.get_figure().savefig('../document/figures/all_site_combined_biasplot.jpg')
ax.get_figure().savefig('../document/figures/all_site_combined_biasplot.pdf')

# TODO rasterize scatterplot backgrounds

In [ ]:
df.describe()

In [ ]:
df.groupby(pd.cut(df.true_elevation,30)).agg({'error':lambda x: np.mean(x**2)**0.5}).rename({'error':'RMS error'})

In [ ]:
tracklines_df = df.groupby([pd.to_datetime(df.delta_time).dt.date,'beam']).mean()

In [ ]:
tracklines_combined.count()

# 178